In [1]:
%load_ext autoreload
%autoreload 2

This notebook illustrates:

- enumeration of DIST-S1 inputs over an MGRS tile
- generation of DIST-S1 products (without confirmation)
- and run the confirmation workflow over the generated products.

In [2]:
from dist_s1_enumerator.dist_enum_inputs import enumerate_dist_s1_workflow_inputs
import pandas as pd
from dist_s1 import run_dist_s1_workflow
from pathlib import Path
from tqdm.auto import tqdm
from dist_s1.workflows import run_sequential_confirmation_of_dist_products_workflow

/Users/cmarshak/miniforge3/envs/dist-s1-env/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Parameters

In [9]:
# Chile Fire Example (Feb 4, 2024)
MGRS_TILE_ID = '19HBD'
START_TIME = '2024-01-01'
STOP_TIME = '2024-03-01'
TRACK_NUMBERS=None
EVENT_ID = 'Chile'

# Enumeration

In [10]:
workflow_inputs = enumerate_dist_s1_workflow_inputs(mgrs_tile_ids=MGRS_TILE_ID,
                                                    track_numbers=TRACK_NUMBERS,
                                                    start_acq_dt=START_TIME,
                                                    stop_acq_dt=STOP_TIME,
                                                    lookback_strategy='multi_window',
                                                    delta_lookback_days=365,
                                                    max_pre_imgs_per_burst=5)
len(workflow_inputs)

Enumerate by MGRS tiles: 100%|█████████████████████████████████████| 1/1 [00:01<00:00,  1.17s/it]


15

In [11]:
workflow_inputs[:]

[{'mgrs_tile_id': '19HBD', 'post_acq_date': '2024-01-04', 'track_number': 91},
 {'mgrs_tile_id': '19HBD', 'post_acq_date': '2024-01-09', 'track_number': 156},
 {'mgrs_tile_id': '19HBD', 'post_acq_date': '2024-01-11', 'track_number': 18},
 {'mgrs_tile_id': '19HBD', 'post_acq_date': '2024-01-16', 'track_number': 91},
 {'mgrs_tile_id': '19HBD', 'post_acq_date': '2024-01-21', 'track_number': 156},
 {'mgrs_tile_id': '19HBD', 'post_acq_date': '2024-01-23', 'track_number': 18},
 {'mgrs_tile_id': '19HBD', 'post_acq_date': '2024-01-28', 'track_number': 91},
 {'mgrs_tile_id': '19HBD', 'post_acq_date': '2024-02-02', 'track_number': 156},
 {'mgrs_tile_id': '19HBD', 'post_acq_date': '2024-02-04', 'track_number': 18},
 {'mgrs_tile_id': '19HBD', 'post_acq_date': '2024-02-09', 'track_number': 91},
 {'mgrs_tile_id': '19HBD', 'post_acq_date': '2024-02-14', 'track_number': 156},
 {'mgrs_tile_id': '19HBD', 'post_acq_date': '2024-02-16', 'track_number': 18},
 {'mgrs_tile_id': '19HBD', 'post_acq_date': '202

# DIST-S1 Product Generation (No Confirmation)

In [12]:
time_series_dir = Path(f'time_series/{EVENT_ID}')

input_data_dir = time_series_dir / 'input_data'
input_data_dir.mkdir(exist_ok=True, parents=True)

scratch_dir = time_series_dir / 'scratch_dir'
scratch_dir.mkdir(exist_ok=True, parents=True)

product_dir_no_confirmation = time_series_dir / 'product_dir'
product_dir_no_confirmation.mkdir(exist_ok=True, parents=True)

In [13]:
for workfolow_input in tqdm(workflow_inputs[:], desc='Generating Products without Confirmation'):
    run_dist_s1_workflow(mgrs_tile_id=workfolow_input['mgrs_tile_id'],
                         post_date=workfolow_input['post_acq_date'], 
                         track_number=workfolow_input['track_number'],
                         memory_strategy='high',
                         input_data_dir=input_data_dir, 
                         dst_dir=scratch_dir,
                         product_dst_dir=product_dir_no_confirmation)

Generating Products without Confirmation:   0%|                           | 0/15 [00:00<?, ?it/s]/Users/cmarshak/miniforge3/envs/dist-s1-env/lib/python3.13/site-packages/dist_s1_enumerator/asf.py:108: UserWarning: No results - please check burst id and availability.
  warn('No results - please check burst id and availability.', category=UserWarning)
/Users/cmarshak/miniforge3/envs/dist-s1-env/lib/python3.13/site-packages/dist_s1_enumerator/asf.py:108: UserWarning: No results - please check burst id and availability.
  warn('No results - please check burst id and availability.', category=UserWarning)


Despeckling and serializing RTC S1 files: 100%|██████████████████| 20/20 [00:12<00:00,  1.58it/s]

Burst disturbance:   0%|                                                   | 0/2 [00:00<?, ?it/s]

Chips Traversed: 100%|████████████████████████████████████████| 661/661 [00:01<00:00, 643.88it/s]
/Users/cmarshak/miniforge3/envs/dist-s1-env/lib/python3.13/site-packages/torch/nn/functional.py

# Confirmation

In [14]:
confirmed_product_dir = time_series_dir / 'confirmed_product_dir'
confirmed_product_dir.mkdir(exist_ok=True, parents=True)

In [15]:
run_sequential_confirmation_of_dist_products_workflow(product_dir_no_confirmation,
                                                      confirmed_product_dir)

Confirming 15 products: 100%|████████████████████████████████████| 15/15 [00:56<00:00,  3.77s/it]
